In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data
from skimage import filters
from skimage import exposure
import cv2
from math import pi

path = "./PHOTOS_MALARIA_VHIR_UPC/fotos_2015_12_01/P_falciparum/Trofozoits/DSCN0083.JPG"

# Original work parameters

## Blur

In [ ]:
img = cv2.imread(path,0)

#resize
img = cv2.resize(img,(640,480))

exp = cv2.imread("./expected_gauss.png",0)

blur = cv2.GaussianBlur(img,(9,9),0)

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 10, forward=True)

blur_tmp = cv2.resize(blur, (exp.shape[1],exp.shape[0]))

ax[0].imshow(blur, cmap='gray')
ax[0].title.set_text("My blur")
ax[1].imshow(exp, cmap='gray')
ax[1].title.set_text("Expected blur")
plt.show()

plt.figure(figsize=(7,7))
plt.imshow(cv2.subtract(exp,blur_tmp),cmap='hot')
plt.title("Difference")
plt.show()

## OTSU (Adaptative)

In [ ]:

exp = cv2.imread("./expected_otsu.png")

block_size = 81
offset = 0.3
thresh_params = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,block_size,offset)
mode = "OTSU Adp block_size=%d offset=%.2f" % (block_size, offset)

block_size = 121
offset = 0.0
thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,block_size,offset)
mode2 = "Most Similar conf block_size=%d offset=%.2f " % (block_size, offset)



#erode
# kernel = np.ones((7, 7), np.uint8)
# thresh = cv2.erode(thresh, kernel)
# #dilate
# kernel = np.ones((5, 5), np.uint8)
# thresh = cv2.dilate(thresh, kernel)
    

fig, ax = plt.subplots(ncols=3, figsize=(20, 20))

ax[0].imshow(thresh_params, cmap='gray')
ax[0].set_title(mode)
ax[1].imshow(thresh, cmap='gray')
ax[1].set_title(mode2)
ax[2].imshow(exp, cmap='gray')
ax[2].set_title('Expected')

[x.axis('off') for x in ax]
plt.show()

In [ ]:
thresh = thresh_params

## Fill holes by contours detection

In [ ]:
# Find contours
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Draw contours
thresh_filled = np.zeros((thresh.shape[0], thresh.shape[1]), dtype=np.uint8)
color = 255
for i in range(len(contours)):
    cv2.drawContours(thresh_filled, contours, i, color, -1, cv2.LINE_8, hierarchy, 0)
    
# Show in a window
plt.figure(figsize=(10,10))
plt.imshow(thresh_filled,cmap='gray')
plt.show()

In [ ]:
#erode
# eroded = thresh_out.copy()
# kernel = np.ones((35, 35), np.uint8)
# for t in range(1):
#     eroded = cv2.erode(eroded, kernel)
    
    
#dilate
#dilated = eroded.copy()
#kernel = np.ones((15, 15), np.uint8)
#for t in range(3):
#    dilated = cv2.dilate(dilated, kernel)


   
#fill
# thresh_fill = thresh.copy()
# h, w = thresh_fill.shape[:2]
# mask = np.zeros((h+2, w+2), np.uint8)
# cv2.floodFill(thresh_fill, mask, (0,0), 255);
# thresh_inv = cv2.bitwise_not(thresh_fill)
# thresh_out = thresh | thresh_inv


# plt.figure(figsize=(10,10))
# plt.imshow(thresh_out, cmap='gray')
# plt.show()

## Connected components

In [ ]:
def connectedComponents(thresh):
    output = cv2.connectedComponentsWithStats(thresh, 4, cv2.CV_32S)
    (numLabels, labels, stats, centroids) = output
    
    # Map component labels to hue val, 0-179 is the hue range in OpenCV
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # Converting cvt to BGR
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    labeled_img_rgb = cv2.cvtColor(labeled_img, cv2.COLOR_BGR2RGB)
    return labeled_img_rgb, numLabels, labels, stats, centroids





In [ ]:
labeled_img_rgb, numLabelsTmp, labelsTmp, statsTmp, centroidsTmp = connectedComponents(thresh_filled)

plt.figure(figsize=(10,10))
plt.imshow(labeled_img_rgb)
plt.show()

## Mark centroids

In [ ]:
def markCentroids(img, numLabels, stats, centroids, estimate=False, cells=(255,255,255)):

    for i in range(1, numLabels):
        # extract the connected component statistics and centroid for
        # the current label
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        #get area
        area = stats[i, cv2.CC_STAT_AREA]
        (cX, cY) = centroids[i]

    #     #get perimeter
    #     cut = thresh_filled[x:w,y:h]
    #     contours,hierarchy = cv2.findContours(cut, 1, 2)

    #     if len(contours) > 0:
    #         cnt = contours[0]
    #         M = cv2.moments(cnt)
    #         perimeter = cv2.arcLength(cnt,True)

    #         compactness = perimeter**2/4*pi*area


        if 500 < area < 1500:
            cv2.circle(img, (int(cX), int(cY)), 8, (0,0,0), -1)
            cv2.circle(img, (int(cX), int(cY)), 5, cells, -1)
        
        if 1500 < area and estimate:
            est = area//1500
            cv2.circle(img, (int(cX)-7, int(cY)), 8, (0,0,0), -1)
            cv2.circle(img, (int(cX)-7, int(cY)), 5, cells, -1)
            cv2.putText(img, str(est), (int(cX), int(cY)+4), cv2.FONT_HERSHEY_SIMPLEX,
                        .5, (0,0,0), 6, cv2.LINE_AA)
            cv2.putText(img, str(est), (int(cX), int(cY)+5), cv2.FONT_HERSHEY_SIMPLEX,
                        .5, (255,255,255), 2, cv2.LINE_AA)
            
            cv2.rectangle(img, (x,y), (x+w,y+h), cells, 2)
 
    return img
    
    
marked_label = labeled_img_rgb.copy()
marked_label = markCentroids(marked_label,numLabelsTmp, statsTmp, centroidsTmp, estimate=True)
    
plt.figure(figsize=(10,10))
plt.imshow(marked_label)
plt.show()

## Break big groups with erosion

In [ ]:
def removeOthersFromCut(img):
    # Connected components with stats.
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)

    max_label, max_size = max([(i, stats[i, cv2.CC_STAT_AREA]) for i in range(1, nb_components)], key=lambda x: x[1])
    
    img2 = np.zeros(output.shape, dtype='uint8')
    img2[output == max_label] = 255
    return img2

OBS: The limits defined that 500 to 1500 isn't enough, some cells, working with original size of image some cells have 2200 of area.

In [ ]:
def erosion(src,size):
    element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * size + 1, 2 * size + 1),
                                       (size, size))
    return cv2.erode(src, element)
    #size -= 2
    #element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * size + 1, 2 * size + 1),
    #                                   (size, size))
    #return cv2.dilate(src,element)
    
def dilatation(src,size):
    element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * size + 1, 2 * size + 1),
                                       (size, size))
    return cv2.dilate(src, element)


def erodeBigGroups(img, numLabels, labels, stats, centroids, size=7):
    
    img2 = img.copy()

    kernel = np.ones((size, size), np.uint8)

    for i in range(1, numLabels):
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        (cX, cY) = centroids[i]

        cutY = y - 1 if y > 0 else y
        cutX = x - 1 if x > 0 else x
        cutXW = x+w+1
        cutYH = y+h+1

        cut = thresh_filled[cutY:cutYH, cutX:cutXW]
        cut = cut.copy()

        if 1500 < area:
            cut_bk = cut.copy()
            cut = removeOthersFromCut(cut)

            #remove the group from original image
            cell_on_original_size = np.zeros( img.shape ,dtype='uint8')
            cell_on_original_size[cutY:cutYH, cutX:cutXW] = cut
            img2[ cell_on_original_size == 255 ] = 0

            #work on group
            #cut = cv2.erode(cut, kernel, iterations=1)
            cut = erosion(cut,7)
            #cut = erosion(cut,15)
            #cut = dilatation(cut,11)

            #return the eroded and marked group to original image
            cell_on_original_size = np.zeros( img.shape ,dtype='uint8')
            cell_on_original_size[cutY:cutYH, cutX:cutXW] = cut
            img2 += cell_on_original_size
    
    return img2



broken_components = erodeBigGroups(thresh_filled, numLabelsTmp, labelsTmp, statsTmp, centroidsTmp, size=9)

plt.figure(figsize=(10,10))
plt.imshow(broken_components, cmap='gray')
plt.show()
        

## Remark centroids

In [ ]:
labeled_img_rgb2, numLabels, labels, stats, centroids = connectedComponents(broken_components)

marked_label = labeled_img_rgb2.copy()
marked_label = markCentroids(marked_label, numLabels, stats, centroids, estimate=True)
    
plt.figure(figsize=(10,10))
plt.imshow(marked_label)
plt.show()

In [ ]:
# A = np.asarray([[[0,0,1],[0,1,0],[1,0,3]],
#      [[0,0,2],[0,1,0],[1,0,0]],
#      [[0,0,2],[0,0,3],[1,0,3]]])



# red, green, blue = A.T # Temporarily unpack the bands for readability

# # Replace white with red... (leaves alpha values alone...)
# white_areas = (red == 0) & (blue == 0) & (green == 1)
# #A[..., :-1][white_areas.T] = (255, 0, 0) # Transpose back needed
# A[white_areas.T] = (255, 0, 0)

# print(white_areas.T)
# print(A)

# #d1, d2, d3 = A.shape
# #A_r = A.reshape(1,9,3)[0]
# #print(A_r)
# #np.unique(A_r, axis=0)

## Shuffle colors

In [ ]:
shape = labeled_img_rgb2.shape
reshapedImg = labeled_img_rgb2.reshape(1,shape[0]*shape[1],3)[0]
colors = np.unique(reshapedImg, axis=0)
colors = np.delete(colors,0,axis=0)

In [ ]:
def random_color():
    color = tuple(np.random.randint(256, size=3))
    if (color[0] < 100 and color[1] < 100 and color[2] < 100):
        return random_color()
    elif (color[0] > 230 and color[1] > 230 and color[2] > 230):
        return random_color()
    else:
        return color

labeled_img_rgb3 = labeled_img_rgb2.copy()

for color in colors:
    newColor = random_color()

    red, green, blue = labeled_img_rgb3.T 
    selected_color = (red == color[0]) & (green == color[1]) & (blue == color[2])
    labeled_img_rgb3[selected_color.T] = newColor

plt.figure(figsize=(10,10))
plt.imshow(labeled_img_rgb3)
plt.show()

# HSV Mask

In [ ]:
img = cv2.imread(path)
img = cv2.resize(img,(640,480))

hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
mask = cv2.inRange(hsv, (110, 0, 0), (150, 255,255))

#erode
kernel = np.ones((3, 3), np.uint8)
mask = cv2.erode(mask, kernel)
#dilate
kernel = np.ones((15, 15), np.uint8)
mask = cv2.dilate(mask, kernel)

par_labeled_rgb, par_numLabels, par_labels, par_stats, par_centroids = connectedComponents(mask)

plt.imshow(mask,cmap='gray')
plt.show()

# Result

In [ ]:

marked = cv2.imread(path)
marked = cv2.resize(marked,(640,480))
marked_label = labeled_img_rgb3.copy()

#MARK CELLS
marked = markCentroids(marked, numLabels, stats, centroids, estimate=True, cells=(0,0,255))
marked_label = markCentroids(marked_label, numLabels, stats, centroids, estimate=True)
        
#MARK PARASITES
for i in range(1, par_numLabels):
    area = par_stats[i, cv2.CC_STAT_AREA]
    (cX, cY) = par_centroids[i]
    
    cv2.circle(marked, (int(cX), int(cY)), 4, (0, 0, 0), -1)
    cv2.circle(marked, (int(cX), int(cY)), 3, (0, 220, 240), -1)
    
    cv2.circle(marked_label, (int(cX), int(cY)), 4, (0, 0, 0), -1)
    cv2.circle(marked_label, (int(cX), int(cY)), 3, (240, 220, 0), -1)
        
    
fig, axes = plt.subplots(ncols=2, figsize=(20, 20))
ax = axes

expR = cv2.imread("./expected_result.png")


ax[0].imshow(cv2.cvtColor(marked, cv2.COLOR_BGR2RGB))
ax[1].imshow(cv2.cvtColor(expR, cv2.COLOR_BGR2RGB))
plt.show()

plt.figure(figsize=(8,8))
plt.imshow(marked_label)
plt.show()